# RicettaBot

Ovvero il RAG che ti fa i calcoli ("calcoli" è un parolone).

### Cosa stiamo per fare

Questo è un progettino che sfrutta LangChain e LLM per risolvere dei semplici calcoli matematici in modo conversazionale. È un RAG perché i calcoli sono descritti in un file separato, su cui agisce un **retriever** che fornisce al LLM le formule più adatte alla richiesta dell'utente.

Utilizziamo due LLM, uno per fare un preprocessing della richiesta, e uno per eseguire il calcolo vero e proprio.
Questo ci da più flessibilità, e ci permette in futuro di fare un eventuale fine-tuning su task più specifici.

Lo schema generale è:

Richiesta -> Retriever + DB Formule -> 2 x LLM -> Risposta

### Strumenti usati

Questa demo utilizza LangChain, Qdrant (pronunciato _quadrant_), Amazon Bedrock per accedere a Claude v2.1, e Cohere per calcolare gli embedding.


## Preparazione

In [14]:
! pip install langchain langchain-community qdrant-client boto3 cohere grandalf

import boto3
import os

os.environ['COHERE_API_KEY'] = "METTI QUI LA TUA CHIAVE"

Reshimming asdf python...


A questo punto prepariamo il client di Amazon Bedrock. Nota bene: è necessario che boto3 sia autenticato e abbia i permessi per accedere a Bedrock!

In [15]:
from langchain_community.llms import Bedrock

def bedrock(model_name="anthropic.claude-v2:1", temperature=0.1):
    client = boto3.client('bedrock-runtime', region_name="us-west-2")
    model = Bedrock(client=client,
                    model_id=model_name,
                    model_kwargs={"temperature": temperature, "max_tokens_to_sample": 1000})
    return model

## Dati

Scriviamo le nostre formule:


In [16]:
formule = """
FORMULA
INTENTO: Calcolo di acqua per fare l pane partendo dalla farina
INPUT: X = kg di farina
OUTPUT: acqua = X * 0.6 litri
END FORMULA


FORMULA
INTENTO: calcolo di zucchero e gelatina per la crema pasticcera
INPUT: X = litri di latte
OUTPUT:
zucchero = X * 100 g
gelatina = X * 5 g
END FORMULA

FORMULA
INTENTO: Calcolo di quanto lievito serve per il pane
INPUT: X = kg di farina
OUTPUT:
lievito = X * 5 g
END FORMULA

FORMULA
INTENTO: Calcolare quanto pangrattato serve per le cotolette
INPUT: X = numero di cotolette
OUTPUT: pangrattato = X * 50 g
END FORMULA

FORMULA
INTENTO: calcolare quanto deve cuocere il risotto
INPUT: X = kg di riso
OUTPUT: Se X < 2, il tempo di cottura è di 15 minuti.
Se X > 2, il tempo di cottura è di 20 minuti.
END FORMULA

FORMULA
INTENTO: Calcolo di quanta farina serve per fare il pane in base al numero di panini
INPUT: X = numero di panini
OUTPUT: farina = X * 200 g
END FORMULA

FORMULA
INTENTO: Calcolare quanti funghi porcini servono per fare il risotto ai funghi porcini partendo dal riso
INPUT: X = kg di riso
OUTPUT: funghi porcini = X * 0.2 kg
END FORMULA

FORMULA
INTENTO: Calcolo di quante mele servono per una torta di mele
INPUT: X = numero di persone che devono mangiare la torta
OUTPUT: mele = X * 2 mele
END FORMULA

FORMULA
INTENTO: Calcolo di quanto pane serve per le persone
INPUT: X = numero di persone
OUTPUT: panini = X * 1,5
END FORMULA
"""

## Database ingestion

Come database usiamo Qdrant in memory, non ci serve un database persistente.

In [17]:
from langchain.vectorstores.qdrant import Qdrant
from langchain_community.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\r", "\t"],
    chunk_size=150,
    chunk_overlap=0
)
chunks = text_splitter.create_documents([formule]) # fatto nella cella precedente

Per salvare i chunk dobbiamo utilizzare degli embedding, che in questo caso sono calcolati da Cohere.

In [18]:
vector_store = Qdrant.from_documents(
                        chunks,
                        location=":memory:",
                        collection_name="formule",
                        embedding=CohereEmbeddings()
                    )

A questo punto il nostro vector store è pronto per essere interrogato.

## Retrieval and generation

Per creare la catena di store e LLM che ci forniranno la risposta utilizziamo langchain. Stiamo facendo un'operazione di prompt engineering, quindi il succo della questione sono i prompt.
Abbiamo diviso le operazioni in due fasi distinte:
- Nella prima fase un LLM legge la richiesta dell'utente e la converte in un json che segue un formato più comprensibile
- Nella seconda fase un altro LLM, un po' più intelligente, legge il json, lo interpreta, e genera la risposta con l'aiuto del retrieval.

Entrambi i prompt sfruttando il few-show per convincere gli LLM a lavorare bene.

In [19]:
parser_prompt_template= """
Sei un agente che legge una RICHIESTA di calcolo per le ricette da parte di un utente
e la traduce in un formato OUTPUT JSON per poter fare il calcolo vero e proprio.

La richiesta è una stringa che contiene una richiesta di calcolo in italiano che riguarda una ricetta.
Se la richiesta non è relativa ad una ricetta devi rispondere con un oggetto JSON vuoto {{}}.

La tua risposta deve essere SOLO un oggetto JSON che contiene la traduzione
in formato json prendendo dalla richiesta le informazioni necessarie,
come oggetto, quantità, ingredienti, persone, ecc.

Le quantità in output che sono richieste dall'utente sono segnate come ???.

Se l'utente chiede quanto lievito serve per fare la pizza, tu devi
rispondere con un oggetto OUTPUT JSON che contiene come "input" gli elementi della richiesta,
e come "output_richiesti" quello che l'utente richiede.
che sono:
- intento: "fare la pizza"
- farina: "2 kg"
E come "output_richiesti" una lista dei valori che l'utente vuole sapere, che in questo caso è:
- "lievito"

Il formato dell'OUTPUT è:
```
{{
    "intento": <intenzione dell'utente>,
    "input": [
         <ingrediente fornito>: <quantità>,
    ],
     "output_richiesti": [
            <ingrediente richiesto 1>,
            <ingrediente richiesto 2>,
            ...
    ]
}}
```
Fornisci in output SOLO il json specificato.

Se non puoi rispondere alla richiesta, rispondi con un oggetto vuoto {{}}.
Non puoi rispondere alla richiesta quando non riesci a capire l'intenzione dell'utente,

Ti faccio qualche esempio:

---- INIZIO ESEMPI ----

RICHIESTA:
Voglio fare le scaloppine per 6 persone, quanta carne mi serve?
OUTPUT JSON:
{{
   "intento": "fare le scaloppine",
   "input": [
       "persone": 6,
   ],
    "output_richiesti": [
        "carne"
   ]
}}

--

RICHIESTA:
Ho 20 persone a cena e voglio fare il pollo arrosto per tutti, quanti polli mi servono e per quanto tempo devo cucinarli in forno?
OUTPUT JSON:
{{
   "intento": "fare il pollo arrosto",
   "input": [
       "persone": 20,
   ],
    "output_richiesti": [
        "polli",
        "tempo di cottura in forno"
   ]
}}

--

RICHIESTA:
Voglio fare lo strudel, quante uova mi servono se il peso totale deve essere 2 kg?
OUTPUT JSON:
{{
   "intento": "fare lo strudel",
   "input": [
       "peso totale": 2,
   ],
    "output_richiesti": [
        "uova"
   ]
}}

--

RICHIESTA:
Devo cucinare la pasta al sugo per i miei 6 amici, di quanti grammi di pasta ho bisogno?
OUTPUT JSON:
{{
   "intento": "fare la pasta al sugo",
   "input": [
       "persone": 6,
   ],
    "output_richiesti": [
        "pasta"
   ]
}}

--


RICHIESTA:
Ho 2 kg di farina e devo fare la pizza, quanto lievito mi serve?
OUTPUT JSON:
{{
    "intento": "fare la pizza",
    "input": [
         "farina": "2 kg",
    ],
     "output_richiesti": [
            "lievito"
    ]
}}


--

RICHIESTA:
Voglio fare 2 litri di salsa di pomodoro, di quanti pomodori e di quanto sale ho bisogno?
OUTPUT JSON:
{{
    "intento": "fare la salsa di pomodoro
    "input": [
         "salsa di pomodoro": "2 litri",
    ],
     "output_richiesti": [
            "pomodori",
            "sale"
    ]
}}

---- FINE ESEMPI ----

RICHIESTA: {input}

OUTPUT JSON:
"""

executor_prompt_template = """
Sei un agente che legge una richiesta di calcolo da una RICHIESTA in formato json
e la risolve, utilizzando le formule fornite nel CONTESTO e applicando i valori della richiesta.

I tuoi input sono la RICHIESTA e il CONTESTO.

Devi trasformare la RICHIESTA in una soluzione applicando le formule del CONTESTO.
Se la richiesta riguarda più formule devi scegliere quelle che ti servono per arrivare al risultato.
Procedi passo per passo, scrivendo il tuo ragionamento in italiano e poi
scrivi il risultato.
NON usare formule che non trovi nel CONTESTO.

Se non ci sono formule nel CONTESTO non puoi rispondere alla richiesta, e devi dire che non puoi rispondere E BASTA, NON DEVI DIRE quali formule ci sono nel contesto.

Se la RICHIESTA è vuota, oppure se mancano input, intento o output, rispondi dicendo che non puoi rispondere perché non hai capito la richiesta.

Il tuo OUTPUT è la spiegazione di quali formule hai scelto, il tuo ragionamento e i tuoi calcoli e poi separatamente il risultato.
Il formato dell'OUTPUT è:
- quali formule hai scelto e perché le hai scelte
- spiegazione dei calcoli
***
<risultato>
***

Ti faccio qualche esempio:
---- INIZIO ESEMPI:

RICHIESTA:
{{
   "intento": "fare le scaloppine",
   "input": [
       "persone": 6,
   ],
    "output_richiesti": [
         "carne"
   ]
}}

CONTESTO:
```
FORMULA
INTENTO: Calcolo della carne per fare le scaloppine
INPUT: X = persone
OUTPUT: carne = X * 200 g
END FORMULA

FORMULA
INTENTO: Calcolo del limone per le scaloppine
INPUT: X = grammi di carne
OUTPUT: limone = X * 1 limone
END FORMULA

FORMULA
INTENTO: Calcolo della farina per la torta
INPUT: X = persone
OUTPUT: farina = X * 100 g
END FORMULA
```

OUTPUT:
l'utente vuole fare la scaloppina per 6 persone e vuole sapere la quantità di carne.
NON uso la formula per calcolare la farina per la torta perché NON è quello che mi serve.
NON uso la formula per calcolare il limone per le scaloppine perché NON è quello che mi viene richiesto negli "output_richiesti".
Uso la formula calcolare quanta carne serve per le scaloppine in base alle persone perché in input ho le persone,
l'intento è fare le scaloppine, e l'output richiesto è carne.

X è il numero di persone
per sapere la carne devo moltiplicare il numero di persone per 200 g
il numero di persone è 6
il calcolo è 6 * 200 g = 1200 g = 1,2 kg

***
Ti servono 1,2 kg di carne per fare le scaloppine per 6 persone
***

-----

RICHIESTA:
{{
    "intento": "fare la pizza"
    "input": [
         "farina": "2 kg",
    ],
     "output_richiesti": [
            "lievito"
    ]
}}

CONTESTO:
```
FORMULA
INTENTO: Calcolo del lievito per fare la pizza
INPUT: X = kg di farina
OUTPUT: lievito = X * 10 g
END FORMULA

FORMULA
INTENTO: Calcolo delle uova per la crema pasticcera
INPUT: X = litri di latte
OUTPUT: uova = X * 4 uova
END FORMULA
```

OUTPUT:
l'utente vuole fare la pizza con 2 kg di farina e vuole sapere la quantità di lievito.
NON uso la formula per calcolare le uova per la crema pasticcera perché non è quello che mi serve.
Uso la formula per calcolare quanto lievito serve per la pizza in base alla farina perché l'input è kg di farina,
l'intento è fare la pizza, e l'output richiesto è lievito.

X è il numero di kg di farina
per sapere la quantità di lievito devo moltiplicare il numero di kg di farina per 10 g
Il numero di kg di farina è 2.
Il calcolo è 2 * 10 g = 20 g di lievito.

****
Ti servono 20 g di lievito per fare la pizza con 2 kg di farina.
****

-----

RICHIESTA:
{{
   "intento": "fare la pasta alla carbonara",
   "input": [
      "persone": 4,
   ],
    "output_richiesti": [
        "uova",
        "pancetta",
        "tempo di cottura",
        "pasta"
    ]
}}

CONTESTO:
```
FORMULA
INTENTO: Calcolo della pasta per fare la carbonara
INPUT: X = persone
OUTPUT: pasta = X * 100 g
END FORMULA

FORMULA
INTENTO: Calcolo delle uova e della pancetta per fare la carbonara
INPUT: X = kg di pasta
OUTPUT: uova = X * 20 uova
        pancetta = X * 100 g
END FORMULA
```

OUTPUT:
La richiesta vuole sapere che quantità usare per la carbonara per 4 persone.
Uso la formula per il calcolo della pasta per fare la carbonara perché tra gli output_richiesti c'è la pasta.
Uso la formula per calcolare le uova e la pancetta per fare la carbonara perché tra gli output_richiesti ci sono uova e pancetta.
NON uso nessun'altra formula.

Calcolo la pasta per 4 persone perché in input ho le persone,
X è il numero di persone.
Per sapere la pasta devo moltiplicare il numero di persone per 100 g
Il numero di persone è 4 quindi il calcolo è 4 * 100 g = 400 g
Mi servono 400 g di pasta per 4 persone

La pasta è stata calcolata nel calcolo precedente, ed è 400 g.
Uso la formula per calcolare le uova e la pancetta usando 400 g di pasta che ho calcolato nel calcolo precedente.
X è il numero di kg di pasta quindi converto 400g in kg.
X = 400 g / 1000 g = 0.4 kg
Per calcolare le uova devo moltiplicare X per 20
Per calcolare la pancetta devo moltiplicare X per 100 g
uova = 0.4 * 20 uova
pancetta = 0.4 * 100 g
Il calcolo è uova = 8 uova e pancetta = 400 g

***
Ti servono 8 uova e 400 g di pancetta per 400 g di pasta per 4 persone.
***

--- FINE ESEMPI

RICHIESTA: {input}

CONTESTO:
```
{formula}
```

OUTPUT:
"""

Questi prompt template usano la notazione `{variabile}` per denotare variabili sostituibili a runtime. Per questa ragione se vogliamo scrivere del json dobbiamo usare ``{{ "due": "Parentesi graffe" }}``.

Ma proseguiamo, definiamo il pezzo di chain che fa il parsing da richiesta a json:

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Cohere
from langchain_community.chat_models.cohere import ChatCohere
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Definiamo prompt e modello per il parser
# Nota: cohere command funziona bene per questo compito
cohere_llm = Cohere(model="command", max_tokens=256, temperature=0.1)
parser_prompt = ChatPromptTemplate.from_template(parser_prompt_template)
parse_model = ChatCohere(llm=cohere_llm)

# Definiamo la catena che legge l'input e arriva ad un output utilizzando
# prompt template e modello
parser_chain = (
    {"input": RunnablePassthrough()}
    | parser_prompt
    | parse_model
    | StrOutputParser()
)

Ora passiamo alla generazione:

In [21]:
executor_prompt = ChatPromptTemplate.from_template(executor_prompt_template)
executor_model = bedrock() # definito sopra sopra

executor_chain = (
    executor_prompt
    | executor_model
    | StrOutputParser()
)

Adesso possiamo unire i vari pezzi!

In [23]:
input = "Quanta farina serve per fare panini per 23 persone?"

parsed_query = parser_chain.invoke({"input": input})
print(parsed_query)

{
    "intento": "fare panini",
    "input": [
        "persone": 23,
    ],
    "output_richiesti": [
        "farina"
    ]
}


E questa è la richiesta in json.

In [24]:
formulas = vector_store.search(parsed_query, k=5, search_type="similarity")

print(formulas)

[Document(page_content='\n\nFORMULA\nINTENTO: Calcolo di quanta farina serve per fare il pane in base al numero di panini\nINPUT: X = numero di panini\nOUTPUT: farina = X * 200 g\nEND FORMULA'), Document(page_content='FORMULA\nINTENTO: Calcolo di quanto pane serve per le persone\nINPUT: X = numero di persone\nOUTPUT: panini = X * 1,5\nEND FORMULA'), Document(page_content='FORMULA\nINTENTO: Calcolo di quanto lievito serve per il pane\nINPUT: X = kg di farina\nOUTPUT:\nlievito = X * 5 g\nEND FORMULA'), Document(page_content='FORMULA\nINTENTO: Calcolare quanto pangrattato serve per le cotolette\nINPUT: X = numero di cotolette\nOUTPUT: pangrattato = X * 50 g\nEND FORMULA'), Document(page_content='FORMULA\nINTENTO: Calcolo di acqua per fare l pane partendo dalla farina\nINPUT: X = kg di farina\nOUTPUT: acqua = X * 0.6 litri\nEND FORMULA')]


Queste sono le formule potrebbero essere utili per la nostra richiesta.

In [25]:
output = executor_chain.invoke({"input": parsed_query, "formula": formulas})

ValueError: Error raised by bedrock service: An error occurred (AccessDeniedException) when calling the InvokeModel operation: You don't have access to the model with the specified model ID.

E questo è il risultato finale, come lo abbiamo richiesto.